In [2]:
%matplotlib inline
!wget https://phm-datasets.s3.amazonaws.com/NASA/5.+Battery+Data+Set.zip -O battery_data.zip
!unzip -o battery_data.zip 
!rm battery_data.zip
!cd "5. Battery Data Set" 
!find "5. Battery Data Set" -name "*.zip" -execdir unzip -o {} \;

--2023-03-19 19:04:03--  https://phm-datasets.s3.amazonaws.com/NASA/5.+Battery+Data+Set.zip
Resolving phm-datasets.s3.amazonaws.com (phm-datasets.s3.amazonaws.com)... 52.216.24.220, 52.217.204.217, 52.217.78.44, ...
Connecting to phm-datasets.s3.amazonaws.com (phm-datasets.s3.amazonaws.com)|52.216.24.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209708670 (200M) [application/zip]
Saving to: ‘battery_data.zip’

battery_data.zip    100%[===================>] 199.99M  96.7MB/s    in 2.1s    

2023-03-19 19:04:05 (96.7 MB/s) - ‘battery_data.zip’ saved [209708670/209708670]

Archive:  battery_data.zip
  inflating: 5. Battery Data Set/1. BatteryAgingARC-FY08Q4.zip  
  inflating: 5. Battery Data Set/2. BatteryAgingARC_25_26_27_28_P1.zip  
  inflating: 5. Battery Data Set/3. BatteryAgingARC_25-44.zip  
  inflating: 5. Battery Data Set/4. BatteryAgingARC_45_46_47_48.zip  
  inflating: 5. Battery Data Set/5. BatteryAgingARC_49_50_51_52.zip  
  inflating: 5. Batter

In [3]:
import datetime
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def load_data(battery):
  mat = loadmat('5. Battery Data Set/' + battery + '.mat')
  print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0]))
  counter = 0
  dataset = []
  capacity_data = []
  
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i]
    if row['type'][0] == 'discharge':
      ambient_temperature = row['ambient_temperature'][0][0]
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      capacity = data[0][0]['Capacity'][0][0]
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_load = data[0][0]['Current_load'][0][j]
        voltage_load = data[0][0]['Voltage_load'][0][j]
        time = data[0][0]['Time'][0][j]
        dataset.append([counter + 1, ambient_temperature, date_time, capacity,
                        voltage_measured, current_measured,
                        temperature_measured, current_load,
                        voltage_load, time])
      capacity_data.append([counter + 1, ambient_temperature, date_time, capacity])
      counter = counter + 1
  print(dataset[0])
  return [pd.DataFrame(data=dataset,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity', 'voltage_measured',
                                'current_measured', 'temperature_measured',
                                'current_load', 'voltage_load', 'time']),
          pd.DataFrame(data=capacity_data,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity'])]
dataset, capacity = load_data('B0005')
# pd.set_option('display.max_columns', 10)
# print(dataset.head())
dataset.describe()

Total data in dataset:  616
[1, 24, datetime.datetime(2008, 4, 2, 15, 25, 41), 1.8564874208181574, 4.191491807505295, -0.004901589207462691, 24.330033885570543, -0.0006, 0.0, 0.0]


,cycle,ambient_temperature,capacity,voltage_measured,current_measured,temperature_measured,current_load,voltage_load,time
count,50285.000000,50285.0,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000
mean,88.125942,24.0,1.560345,3.515268,-1.806032,32.816991,1.362700,2.308406,1546.208924
std,45.699687,0.0,0.182380,0.231778,0.610502,3.987515,1.313698,0.800300,906.640295
min,1.000000,24.0,1.287453,2.455679,-2.029098,23.214802,-1.998400,0.000000,0.000000
25%,50.000000,24.0,1.386229,3.399384,-2.013415,30.019392,1.998000,2.388000,768.563000
50%,88.000000,24.0,1.538237,3.511664,-2.012312,32.828944,1.998200,2.533000,1537.031000
75%,127.000000,24.0,1.746871,3.660903,-2.011052,35.920887,1.998200,2.690000,2305.984000
max,168.000000,24.0,1.856487,4.222920,0.007496,41.450232,1.998400,4.238000,3690.234000


#creds to https://www.kaggle.com/rajeevsharma993